# Homework 7: Encrypted QAP

Given an R1CS, you should transform it into a QAP (you can use the code from the RareSkills ZK Book for this).

This will produce polynomials U, V, W, and HT (see the notation in the book).

Do an encrypted evaluation of each of these polynomials, this will result in:

$$
\begin{align}
eval(U) &= [A]_1\\
eval(V) &= [B]_2,\\ 
eval(W) &= [C']_1,\\
eval(HT) &= [HT]_1
\end{align}
$$

Create
$$
\left[C\right]=\left[C'\right]_1+\left[HT\right]_1
$$

Then verify that
$$
\text{pairing}([A]_1,[B]_2)-\text{pairing}([C]_1,[G]_2) = 0
$$

Do the verification on chain.

Your code should be able to start with an arbitrary R1CS and compute the three elliptic curve points:
$$
[A]_1, [B]_2, [C]_1
$$
for which the verifier will check if the pairing is correct.

## Answer

We can start with the same problem from week 6, problem 4:

```python
import numpy as np

# Define the matrices
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick values for x and y
x = 1
y = 2

# this is our original formula
# the witness vector with the intermediate variables inside
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3
v1 = 3*x*x
v2 = v1 * y
s = np.array([1, out, x, y, v1, v2])
```

First we start with the conversion of the R1CS matrices into polynomials:

```python
import numpy as np
from scipy.interpolate import lagrange

# Define the matrices
# Renaming them to match the latest notation
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick values for x and y
x = 1
y = 2

# this is our original formula
# the witness vector with the intermediate variables inside
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3
v1 = 3*x*x
v2 = v1 * y
s = np.array([1, out, x, y, v1, v2])

"""
Interpolates a vector v containing the y coordinates over x = [0, 1, 2].
"""
def phi(v):
    return lagrange(range(len(v)), v)

"""
Converts an R1CS matrix to a polynomial by multiplying each column by the witness vector and adding the result.
"""
def r1cs_to_polynomial(M, v):
    A = [0 * len(v)]
    for j in range(len(M[0])):
        A = A + (M[:,j] * v[j])
    return phi(A)
    # Alternatively it could be implemented as:
    # p = np.poly1d([0])
    # for j in range(len(M[0])):
    #     p = p + phi(M[:,j] * v[j])
    # return p

u = r1cs_to_polynomial(L, s)
v = r1cs_to_polynomial(R, s)
w = r1cs_to_polynomial(O, s)
t = np.poly1d([1, -3, 2, 0]) # t(x) is `x*(x-1)*(x-2)`

(h, r) = ((u * v) - w) / t
# Sanity check: the remainder of the division should be zero.
assert r == np.poly1d([0])
```

Next we need to encrypt the params using ECC

```python
from py_ecc.bn128 import curve_order, G1, G2, add, multiply, eq, pairing

"""
`py_ecc.bn128.multiply` cannot deal with negative numbers, so we need to take its modular inverse if `x < 0`.
"""
def normalized_multiply(G, x):
    return multiply(G, x if x >= 0 else curve_order + x)

A1 = normalized_multiply(G1, u(x))
B2 = normalized_multiply(G2, v(x))
C1 = add(normalized_multiply(G1, w(x)), normalized_multiply(G1, (h * t)(x)))

assert eq(pairing(B2, A1), pairing(G2, C1))
```

Below there is a full working example:

In [24]:
import numpy as np
from scipy.interpolate import lagrange
from py_ecc.bn128 import curve_order, G1, G2, add, multiply, eq, pairing

# Define the matrices
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick values for x and y
x = 1
y = 2

# this is our original formula
# the witness vector with the intermediate variables inside
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3
v1 = 3*x*x
v2 = v1 * y
s = np.array([1, out, x, y, v1, v2])

"""
Interpolates a vector v containing the y coordinates over x = [0, 1, 2].
"""
def phi(v):
    return lagrange(range(len(v)), v)

"""
Converts an R1CS matrix to a polynomial by multiplying each column by the witness vector and adding the result.
"""
def r1cs_to_polynomial(M, v):
    A = [0 * len(v)]
    for j in range(len(M[0])):
        A = A + (M[:,j] * v[j])
    return phi(A)

u = r1cs_to_polynomial(L, s)
v = r1cs_to_polynomial(R, s)
w = r1cs_to_polynomial(O, s)
t = np.poly1d([1, -3, 2, 0]) # t(x) is `x*(x-1)*(x-2)`

(h, r) = ((u * v) - w) / t
# Sanity check: the remainder of the division should be zero.
assert r == np.poly1d([0])

"""
`py_ecc.bn128.multiply` cannot deal with negative numbers, so we need to take its modular inverse if `x < 0`.
"""
def normalized_multiply(G, x):
    return multiply(G, x if x >= 0 else curve_order + x)

A1 = normalized_multiply(G1, u(x))
B2 = normalized_multiply(G2, v(x))
C1 = add(normalized_multiply(G1, w(x)), normalized_multiply(G1, (h * t)(x)))

assert eq(pairing(B2, A1), pairing(G2, C1))

# We assume τ to be 5
# tau = 5

# """
# Returns the `n` powers of `t` multiplied by the generator point G over an elliptic curve, like:
# [t**(n-1).G, t**(n-2).G, ..., t**0.G]
# """
# def powers_of_tau(G, t, n):
#     return np.array([multiply(G, t**i) for i in reversed(range(n))])

# T1 = powers_of_tau(G1, tau, u.order + v.order)
# T2 = powers_of_tau(G2, tau, u.order + v.order)